In [1]:
import pandas as pd

In [7]:
lyrics_file = pd.read_csv("lyrics.csv")

In [3]:
df.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [4]:
df.shape

(362237, 6)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

import pandas as pd
from sklearn.feature_extraction import stop_words
import re
import string

In [8]:
lyrics_file = lyrics_file[(lyrics_file.genre != "Not Available") 
                          & (lyrics_file.genre != "Other") 
                          & lyrics_file.lyrics.notnull()
                         & lyrics_file.song.notnull()]

In [9]:

def clean_lyrics(text):
    text = re.sub('\n', ' ', text)  # removes new lines
    text = re.sub('\d', '', text)  # removes numbers
    text = re.sub('\t', '', text)  # removes tabs
    text = re.sub('[^A-Za-z ]', '', text)  # removes special character
    text = re.sub('\[Verse\s*\d*:]', ' ', text)  # remove verse indicators
    text = re.sub('\s+', ' ', text)
    words = text.split(" ")
    words = [w for w in words if len(w) > 2]  # removes a, an, to, at, be, ...
    return words

In [11]:
target_ly = lyrics_file[["genre"]]
feature_ly = lyrics_file[["lyrics"]]

train_data, test_data, train_target, test_target = train_test_split(feature_ly,
                                                                    target_ly)

In [12]:
tfidf = TfidfVectorizer(stop_words='english',
                       tokenizer = clean_lyrics)

# we fit the train data
tf_features = tfidf.fit_transform(train_data.lyrics)

# and we transform the test data
test_features = tfidf.transform(test_data.lyrics)

In [13]:
# we fit a vectorizer
# we create the vectorizer
tfidf = TfidfVectorizer(stop_words='english',
                       tokenizer = clean_lyrics)

# we fit the train data
tf_features = tfidf.fit_transform(train_data.lyrics)

# and we transform the test data
test_features = tfidf.transform(test_data.lyrics)

In [15]:
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(tf_features)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Top terms per cluster:
Cluster 0:
 instrumental
 lyrics
 song
 album
 score
 track
 intro
 september
 piece
 music
Cluster 1:
 like
 time
 just
 life
 que
 come
 away
 love
 world
 night
Cluster 2:
 love
 know
 baby
 dont
 heart
 just
 ill
 want
 youre
 like
Cluster 3:
 dont
 know
 just
 like
 got
 youre
 want
 baby
 time
 say


Prediction
